# Tokenize Openwebtext

## Download Dataset

In [1]:
import datasets
from datasets import load_dataset
dataset = load_dataset("openwebtext")

/common/home/zp128/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset openwebtext (/research/projects/zp128/dataset_tokenizedGbt2/downloaded_dataset/openwebtext/plain_text/1.0.0/85b3ae7051d2d72e7c5fdf6dfb462603aaa26e9ed506202bf3a24d261c6c40a1)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.37s/it]


## Load Tokenizer and Get Vocab Size

In [2]:
#show how many documents in this dataset
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
print("Gpt2 Tokenizer Vocab Size:"+ str(tokenizer.vocab_size))
print(tokenizer.is_fast)

def tokenize_map_function(items):
    # iterate each item in a batch
    # print(type(items["text"]))
    global tokenizer
    return tokenizer(items["text"],max_length=10000)

tokenized_dataset = dataset.map(tokenize_map_function, batched = True)

print(tokenized_dataset.column_names)

Parameter 'function'=<function tokenize_map_function at 0x7f6b2c0cd1f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Gpt2 Tokenizer Vocab Size:50257
True


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 8014/8014 [2:33:23<00:00,  1.15s/ba]


{'train': ['text', 'input_ids', 'attention_mask']}


In [4]:
# save this dataset for future usage
tokenized_dataset.save_to_disk("./tokenized_dataset")

In [8]:
# load back
tokenized_dataset.load_from_disk("./tokenized_dataset")

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 8013769
    })
})

In [10]:
tokenized_dataset = tokenized_dataset['train']

## Write tokenized Dataset to the binary file

In [14]:
import numpy as np
encodedFile = "./openwebtext_gpt2.bin"
encodedSeqF = open(encodedFile,'wb')

def write2BinaryFile(items):
    global encodedSeqF
#     written_bytes = b''
    for tokens in items['input_ids']:
        tokens_len =int(len(tokens))
        # write its length
#         written_bytes += tokens_len.to_bytes(4,byteorder='little',signed=True)
#         written_bytes +=np.array(tokens,dtype=np.uint32).tobytes() 
        encodedSeqF.write(tokens_len.to_bytes(4,byteorder='little',signed=True))
        # write the list of tokens
        encodedSeqF.write(np.array(tokens,dtype=np.uint32).tobytes())
#     encodedSeqF.write(written_bytes) 

# use map function of dataset to write each tokens into binary file
tokenized_dataset = tokenized_dataset.map(write2BinaryFile, batched = True)
encodedSeqF.close()


# cnt =0 
# for tokens in tokenized_dataset['train']['input_ids']:
#     tokens_len =int(len(tokens))
#     encodedSeqF.write(tokens_len.to_bytes(4,byteorder='little',signed=True))
#     cnt += 1
#     for i in range(0,tokens_len):
#         encodedSeqF.write((tokens[i]).to_bytes(4,byteorder='little',signed=True))

#     if cnt%10000 == 0:
#         print("Current Written text :"+str(cnt))

100%|███████████████████████████| 8014/8014 [2:10:39<00:00,  1.02ba/s]


## Debug for tokenizedPile

In [4]:
import datasets
from datasets import load_from_disk
tokenized_dataset = load_from_disk("./tokenized_dataset/tokenized_pile")


/common/home/zp128/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# it = iter(tokenized_dataset)s
for i in range(0,79491):
    tmp_tokens = next(it)
    if(len(tmp_tokens['input_ids'])<=5):
        print(len(tmp_tokens['input_ids']))
#     print(len(tmp_tokens['input_ids']))

0
1
2
5
2
5
3
4
0
2
3
2
3
4
4
1
4
3
5
2
3
3
3
0
4
0
2
3
1
4
5
3
2
4


In [1]:
from datasets import load_dataset
dataset = load_dataset("glue", "mrpc", split="train")

/common/home/zp128/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset glue (/common/home/zp128/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
dataset = dataset.skip(3)
it = iter(dataset)
next(it)

AttributeError: 'Dataset' object has no attribute 'skip'

In [33]:
import numpy as np
encodedFile = "./testtest.bin"
encodedSeqF = open(encodedFile,'wb')
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

buffer = []
for i in range(0, dataset.num_rows):
    text = next(it)['sentence1']
    tokens = tokenizer(text,max_length=100000)
    tokens = tokens['input_ids']
    buffer.append(len(tokens))
    buffer.extend(tokens)
    
    if len(buffer) > 10000:
        encodedSeqF.write(np.array(buffer,dtype = np.uint32).tobytes())
        # write the list of tokens
        buffer = []

encodedSeqF.close()        

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [37]:
import os
os.system("keep-job 64")


0

# Get the vocabulary of the tokenizer

In [1]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

/common/home/zp128/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocab = tokenizer.get_vocab()

In [3]:
keys = [[k,v] for k, v in vocab.items() if v == 220]
print(keys)


[['Ġ', 220]]
